In [77]:
import torch
import numpy as np

In [78]:
class Dual:

    def __init__(self, real, dual):
        """
        :param real: int or float
        :param dual: dict (str: float)
        """
        if not isinstance(real, (int, float)) or not isinstance(dual, dict):
            raise TypeError("Wrong type of argument(s)!")
        self.real = float(real)
        self.dual = dual

    def __repr__(self) -> str:
        from pprint import pformat
        return pformat(vars(self), indent=4, width=1, sort_dicts=False)

    def __add__(self, other):
        if not isinstance(other, Dual):
            raise TypeError("Undefined operation!")
        new_real = self.real + other.real
        new_dual = dict(self.dual)
        for var in other.dual:
            if var in new_dual:
                new_dual[var] += other.dual[var]
            else:
                new_dual[var] = other.dual[var]
        return Dual(new_real, new_dual)

    def __sub__(self, other):
        if not isinstance(other, Dual):
            raise TypeError("Undefined operation!")
        new_real = self.real - other.real
        new_dual = dict(self.dual)
        for var in other.dual:
            if var in new_dual:
                new_dual[var] -= other.dual[var]
            else:
                new_dual[var] = -other.dual[var]
        return Dual(new_real, new_dual)

    def __mul__(self, other):
        if not isinstance(other, Dual):
            raise TypeError("Undefined operation!")
        new_real = self.real * other.real
        new_dual = {var: self.dual[var] * other.real for var in self.dual}
        for var in other.dual:
            if var in new_dual:
                new_dual[var] += other.dual[var] * self.real
            else:
                new_dual[var] = other.dual[var] * self.real
        return Dual(new_real, new_dual)


def tanh(dual: Dual):
    new_real = np.tanh(dual.real)
    new_dual = {var: dual.dual[var] * (1 - np.tanh(dual.real) ** 2) for var in dual.dual}
    return Dual(new_real, new_dual)

In [79]:
d1 = Dual(2.8, {'x': 3.7, 'y': 8.7})
d2 = Dual(4.9, {'x': 5.3, 'y': 7.2})
print(d1 + d2)
print(d1 - d2)
print(d1 * d2)
print(tanh(d1))

{   'real': 7.7,
    'dual': {   'x': 9.0,
                'y': 15.899999999999999}}
{   'real': -2.1000000000000005,
    'dual': {   'x': -1.5999999999999996,
                'y': 1.4999999999999991}}
{   'real': 13.72,
    'dual': {   'x': 32.97,
                'y': 62.790000000000006}}
{   'real': 0.992631520201128,
    'dual': {   'x': 0.054325860881831126,
                'y': 0.12773918639781912}}
